In [ ]:
# @title 1. Install Libraries (Fixed for MoviePy 1.0.3)
# 1. Install the main AI libraries
!pip install -q --upgrade google-generativeai edge-tts torch torchaudio torchvision diffusers transformers accelerate

# 2. Force install the specific version of MoviePy that supports .editor
!pip install moviepy==1.0.3

!pip install -q groq

# 3. Install ImageMagick (Required for text/video processing)
!apt install imagemagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

print("✅ Installation Complete - MoviePy 1.0.3 Restored")

In [ ]:
# @title Bible Story Bot (Full Pipeline)
import os
import asyncio
import json
import torch
import edge_tts
from groq import Groq
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler
from moviepy.editor import *
from google.colab import drive
from deep_translator import GoogleTranslator
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# --- CONFIG ---
GROQ_API_KEY = "YOUR_GROQ_KEY_HERE"
DRIVE_FOLDER = "/content/drive/MyDrive/BibleStories"
TOPIC = "The Story of Joseph"

# --- SETUP ---
drive.mount('/content/drive')
os.makedirs(DRIVE_FOLDER, exist_ok=True)

# Load Models
pipe = StableDiffusionXLPipeline.from_pretrained(
    "Lykon/dreamshaper-xl-1-0", 
    torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
)
pipe.enable_model_cpu_offload()

# --- 1. GENERATE SCRIPT ---
def generate_script(topic):
    client = Groq(api_key=GROQ_API_KEY)
    glossary = "God -> దేవుడు (Devudu)\nJoseph -> యోసేపు (Yosepu)" # Add full list here
    
    prompt = f"""
    You are a Telugu Grandmother. Tell the story of {topic}.
    {glossary}
    Output JSON with 'scenes' (10), 'lesson' (1), 'blessing' (1).
    - narration: Telugu
    - visual: English
    """
    
    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[{"role":"system", "content":prompt}],
        response_format={"type": "json_object"}
    )
    return json.loads(completion.choices[0].message.content)

# --- 2. ASSETS & VIDEO ---
async def process_video(script):
    clips = []
    translator = GoogleTranslator(source='auto', target='en')
    
    segments = script['scenes'] + [script['lesson']] + [script['blessing']]
    
    for i, seg in enumerate(segments):
        # Audio
        audio_path = f"audio_{i}.mp3"
        await edge_tts.Communicate(seg['narration'], "te-IN-ShrutiNeural").save(audio_path)
        
        # Image (Safety Net)
        try: visual = translator.translate(seg['visual'])
        except: visual = seg['visual']
        
        img_path = f"image_{i}.png"
        pipe(prompt=f"3D disney style, {visual}", height=1024, width=1024).images[0].save(img_path)
        
        # Clip
        audio = AudioFileClip(audio_path)
        clip = ImageClip(img_path).set_duration(audio.duration+0.5).set_audio(audio)
        clips.append(clip)
        
    final = concatenate_videoclips(clips, method="compose")
    out_path = f"{DRIVE_FOLDER}/{TOPIC}.mp4"
    final.write_videofile(out_path, fps=24)
    return out_path

# --- 3. UPLOAD ---
def upload(file_path):
    token_path = f"{DRIVE_FOLDER}/token.json"
    creds = Credentials.from_authorized_user_file(token_path)
    youtube = build('youtube', 'v3', credentials=creds)
    
    body = {
        'snippet': {'title': f"{TOPIC} (Telugu)", 'categoryId': '22'},
        'status': {'privacyStatus': 'public', 'selfDeclaredMadeForKids': False}
    }
    youtube.videos().insert(part="snippet,status", body=body, media_body=MediaFileUpload(file_path)).execute()

# Run
script = generate_script(TOPIC)
video_path = await process_video(script)
upload(video_path)